<a href="https://colab.research.google.com/github/yukismd/DL_for_ImageData_and_Finetuning/blob/main/text_classification_hatespeech/PT/bert-large-japanese-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
MODEL = 'tohoku-nlp/bert-large-japanese-v2'

In [1]:
!pwd

/content


In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls

drive  sample_data


In [31]:
!ls drive/MyDrive/experiment_results

hs_txt_classification


In [5]:
SAVE_DIR = 'drive/MyDrive/experiment_results/hs_txt_classification'

In [6]:
!pip install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00


In [7]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [8]:
!pip install fugashi unidic_lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic_lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=61bfd37ba7c986d3be536c2a5320e058f2303381d57fdd5770dd0de7218f741c
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic_lite


In [9]:
import datasets
print(datasets.__version__)
import transformers
print(transformers.__version__)
from transformers import AutoTokenizer
import evaluate
print(evaluate.__version__)

import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import json

2.19.1
4.41.1
0.4.2


## 学習データ, テストデータ

In [10]:
df = pd.read_csv('https://sample-data-open.s3.ap-northeast-1.amazonaws.com/text_classification_hatespeech/hatespeech_train.csv')
print(df.shape)

(4656, 5)


In [11]:
df_train = df[df['hold'] != 0]    # 学習データ
df_valid = df[df['hold'] == 0]    # 検証データ
df_train.shape, df_valid.shape

((3759, 5), (897, 5))

In [12]:
ds_train = datasets.Dataset.from_pandas(df_train[['text','label']], preserve_index=False)
ds_valid = datasets.Dataset.from_pandas(df_valid[['text','label']], preserve_index=False)

In [13]:
df_test = pd.read_csv('https://sample-data-open.s3.ap-northeast-1.amazonaws.com/text_classification_hatespeech/hatespeech_test.csv')
print(df_test.shape)

(600, 4)


In [14]:
ds_test = datasets.Dataset.from_pandas(df_test[['text','label']], preserve_index=False)

## トークナイザー

In [16]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

BertJapaneseTokenizer(name_or_path='tohoku-nlp/bert-large-japanese-v2', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [17]:
def tokenize_function(examples):
    return tokenizer(examples["text"],
                     truncation=True,
                     max_length=128)    # 学習データの文長が短いので128に設定

In [18]:
ds_tkn_train = ds_train.map(tokenize_function, batched=True)
ds_tkn_valid = ds_valid.map(tokenize_function, batched=True)
ds_tkn_train, ds_tkn_valid

Parameter 'function'=<function tokenize_function at 0x7ea05d2f1360> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/3759 [00:00<?, ? examples/s]

Map:   0%|          | 0/897 [00:00<?, ? examples/s]

(Dataset({
     features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 3759
 }),
 Dataset({
     features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 897
 }))

In [19]:
# textカラムの削除
ds_tkn_train = ds_tkn_train.remove_columns(['text'])
ds_tkn_valid = ds_tkn_valid.remove_columns(['text'])

# labelカラムをlabelsへリネーム
ds_tkn_train = ds_tkn_train.rename_column('label', 'labels')
ds_tkn_valid = ds_tkn_valid.rename_column('label', 'labels')

In [20]:
ds_tkn_test = ds_test.map(tokenize_function, batched=True)
ds_tkn_test

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 600
})

In [21]:
# textカラムの削除
ds_tkn_test = ds_tkn_test.remove_columns(['text'])

# labelカラムをlabelsへリネーム
ds_tkn_test = ds_tkn_test.rename_column('label', 'labels')

## モデル

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL,
                                                           num_labels=2)

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tohoku-nlp/bert-large-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 学習

In [28]:
metric = evaluate.load("f1")

# 精度評価関数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [36]:
def set_training():
  print('N_EPOCH: ', N_EPOCH)
  print('BATCH_SIZE: ', BATCH_SIZE)
  print('LR', LR)

  trainer_args = TrainingArguments(
    seed=123,
    output_dir=SAVE_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    #save_strategy="epoch",
    save_steps=1e6, # saveのステップを大きくしてここではモデルの保存を行わないようにする
    log_level="critical",
    num_train_epochs=N_EPOCH,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=12,
    save_total_limit=1,
    fp16=True,
    remove_unused_columns=False,
    report_to="none"
    )

  trainer = Trainer(
    model=model,
    args=trainer_args,
    tokenizer=tokenizer,
    train_dataset=ds_tkn_train,
    eval_dataset=ds_tkn_valid,
    compute_metrics=compute_metrics,
    )

  return trainer

In [35]:
def save_res_valid(trainer):
  res_json = trainer.evaluate()
  print(res_json)
  save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
  print(save_file)
  with open(save_file, 'w') as f:
      json.dump(res_json, f, indent=2)
  return res_json

def save_res_test(trainer):
  test_preds = trainer.predict(ds_tkn_test)
  test_metric = test_preds.metrics
  print(test_metric)
  save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
  print(save_file)
  with open(save_file, 'w') as f:
      json.dump(test_metric, f, indent=2)
  return test_metric

In [37]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.000003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

print('----------  Start Training  ----------')
experiment_trainer = set_training()
experiment_trainer.train()

print('----------  Res on Valid  ----------')
res_val = save_res_valid(experiment_trainer)

print('----------  Res on Test  ----------')
res_test = save_res_test(experiment_trainer)

#res_val, res_test

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06
----------  Start Training  ----------
N_EPOCH:  3
BATCH_SIZE:  16
LR 3e-06


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'loss': 0.0895, 'grad_norm': 0.03305274620652199, 'learning_rate': 2.0085106382978724e-06, 'epoch': 1.0}
{'eval_loss': 0.15946632623672485, 'eval_f1': 0.6407766990291262, 'eval_runtime': 3.0944, 'eval_samples_per_second': 289.876, 'eval_steps_per_second': 24.237, 'epoch': 1.0}
{'loss': 0.0492, 'grad_norm': 16.001604080200195, 'learning_rate': 1.0085106382978723e-06, 'epoch': 2.0}
{'eval_loss': 0.17619889974594116, 'eval_f1': 0.6603773584905661, 'eval_runtime': 3.0325, 'eval_samples_per_second': 295.793, 'eval_steps_per_second': 24.732, 'epoch': 2.0}
{'loss': 0.0239, 'grad_norm': 0.01853679120540619, 'learning_rate': 8.51063829787234e-09, 'epoch': 3.0}
{'eval_loss': 0.17803332209587097, 'eval_f1': 0.6666666666666666, 'eval_runtime': 3.0974, 'eval_samples_per_second': 289.596, 'eval_steps_per_second': 24.214, 'epoch': 3.0}
{'train_runtime': 236.6582, 'train_samples_per_second': 47.651, 'train_steps_per_second': 2.979, 'train_loss': 0.054201863336224926, 'epoch': 3.0}
----------  Res on 

({'eval_loss': 0.17803332209587097,
  'eval_f1': 0.6666666666666666,
  'eval_runtime': 3.0403,
  'eval_samples_per_second': 295.039,
  'eval_steps_per_second': 24.669,
  'epoch': 3.0},
 {'test_loss': 0.24746118485927582,
  'test_f1': 0.6315789473684211,
  'test_runtime': 2.0278,
  'test_samples_per_second': 295.886,
  'test_steps_per_second': 24.657})

In [38]:
!ls drive/MyDrive/experiment_results/hs_txt_classification

checkpoint-940
TEST__bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06.json
VALID__bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06.json


In [39]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.000001  # 1e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

print('----------  Start Training  ----------')
experiment_trainer = set_training()
experiment_trainer.train()

print('----------  Res on Valid  ----------')
res_val = save_res_valid(experiment_trainer)

print('----------  Res on Test  ----------')
res_test = save_res_test(experiment_trainer)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch16_Lr1e-06
----------  Start Training  ----------
N_EPOCH:  3
BATCH_SIZE:  16
LR 3e-06


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'loss': 0.0124, 'grad_norm': 0.008162029087543488, 'learning_rate': 6.695035460992907e-07, 'epoch': 1.0}
{'eval_loss': 0.21704255044460297, 'eval_f1': 0.6324786324786325, 'eval_runtime': 3.0989, 'eval_samples_per_second': 289.46, 'eval_steps_per_second': 24.202, 'epoch': 1.0}
{'loss': 0.0053, 'grad_norm': 4.669007301330566, 'learning_rate': 3.3758865248226946e-07, 'epoch': 2.0}
{'eval_loss': 0.23174817860126495, 'eval_f1': 0.6140350877192983, 'eval_runtime': 3.0438, 'eval_samples_per_second': 294.694, 'eval_steps_per_second': 24.64, 'epoch': 2.0}
{'loss': 0.004, 'grad_norm': 0.00402480736374855, 'learning_rate': 4.25531914893617e-09, 'epoch': 3.0}
{'eval_loss': 0.22113452851772308, 'eval_f1': 0.6666666666666666, 'eval_runtime': 3.1667, 'eval_samples_per_second': 283.258, 'eval_steps_per_second': 23.684, 'epoch': 3.0}
{'train_runtime': 235.4081, 'train_samples_per_second': 47.904, 'train_steps_per_second': 2.995, 'train_loss': 0.007246097436187961, 'epoch': 3.0}
----------  Res on Vali

In [40]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.000001  # 1e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

print('----------  Start Training  ----------')
experiment_trainer = set_training()
experiment_trainer.train()

print('----------  Res on Valid  ----------')
res_val = save_res_valid(experiment_trainer)

print('----------  Res on Test  ----------')
res_test = save_res_test(experiment_trainer)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch16_Lr1e-06
----------  Start Training  ----------
N_EPOCH:  3
BATCH_SIZE:  16
LR 3e-06


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'loss': 0.0038, 'grad_norm': 0.0009768233867362142, 'learning_rate': 6.695035460992907e-07, 'epoch': 1.0}
{'eval_loss': 0.2603462040424347, 'eval_f1': 0.6534653465346535, 'eval_runtime': 3.0414, 'eval_samples_per_second': 294.931, 'eval_steps_per_second': 24.66, 'epoch': 1.0}
{'loss': 0.0008, 'grad_norm': 0.06592584401369095, 'learning_rate': 3.3758865248226946e-07, 'epoch': 2.0}
{'eval_loss': 0.2854660749435425, 'eval_f1': 0.6306306306306306, 'eval_runtime': 2.9649, 'eval_samples_per_second': 302.536, 'eval_steps_per_second': 25.296, 'epoch': 2.0}
{'loss': 0.0017, 'grad_norm': 0.0008126851171255112, 'learning_rate': 4.25531914893617e-09, 'epoch': 3.0}
{'eval_loss': 0.2926115095615387, 'eval_f1': 0.6306306306306306, 'eval_runtime': 4.6553, 'eval_samples_per_second': 192.685, 'eval_steps_per_second': 16.111, 'epoch': 3.0}
{'train_runtime': 232.6702, 'train_samples_per_second': 48.468, 'train_steps_per_second': 3.03, 'train_loss': 0.0020984541651204967, 'epoch': 3.0}
----------  Res on 